<a href="https://colab.research.google.com/github/xerxes786/Random-Machine-Learning-Code/blob/main/Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
train = '/content/drive/MyDrive/Sports data set/train'
test = '/content/drive/MyDrive/Sports data set/test'
valid = '/content/drive/MyDrive/Sports data set/valid'


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
filepaths_train = list(glob.glob(train+'/**/*.jpg'))
filepaths_test = list(glob.glob(test+'/**/*.jpg'))

In [ ]:
import pandas as pd
import os
def proc_img(filepath):
 
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepath))

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    df = pd.concat([filepath, labels], axis=1)
    df = df.sample(frac=1).reset_index(drop = True)
  
    return df
train_df = proc_img(filepaths_train)
test_df = proc_img(filepaths_test)

In [ ]:
print(f'Number of training pictures: {train_df.shape[0]}')
print(f'Number of test pictures: {test_df.shape[0]}')

Number of training pictures: 10428
Number of test pictures: 365


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    subset='training',
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0,
    subset='validation',
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 8343 validated image filenames belonging to 73 classes.
Found 2085 validated image filenames belonging to 73 classes.
Found 365 validated image filenames belonging to 73 classes.


In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(73, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
ACCURACY_THRESHOLD = 1
from keras.callbacks import Callback
class myCallback(Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True

In [ ]:
callbacks = myCallback()
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=5,
    callbacks=[callbacks]
)

Epoch 1/5
261/261 [==============================] - 345s 1s/step - loss: 1.7169 - accuracy: 0.5708 - val_loss: 0.6710 - val_accuracy: 0.8230
Epoch 2/5
261/261 [==============================] - 323s 1s/step - loss: 0.4898 - accuracy: 0.8550 - val_loss: 0.5785 - val_accuracy: 0.8312
Epoch 3/5
261/261 [==============================] - 319s 1s/step - loss: 0.2685 - accuracy: 0.9215 - val_loss: 0.4952 - val_accuracy: 0.8547
Epoch 4/5
261/261 [==============================] - 333s 1s/step - loss: 0.1565 - accuracy: 0.9582 - val_loss: 0.5089 - val_accuracy: 0.8494
Epoch 5/5
261/261 [==============================] - 312s 1s/step - loss: 0.1018 - accuracy: 0.9743 - val_loss: 0.4970 - val_accuracy: 0.8643


In [ ]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

    Test Loss: 0.18945
Test Accuracy: 94.25%
